#### 라이브러리 임포트

In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
)
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks

#### 데이터셋 로딩

In [77]:
# 버전1
# 제미나이 임베딩 벡터 데이터셋 로딩

# import pandas as pd

# df_processed = pd.read_json(
#     "Dataset/review_business_5up_with_embedded_vector.jsonl",
#     lines=True,
# )

In [78]:
# 버전2
# 데이터가 너무 커 테스트로 10000개만 읽어옴

import pandas as pd

json_reader = pd.read_json(
    "Dataset/review_business_5up_with_embedded_vector.jsonl",
    lines=True,
    chunksize=10000,
)

df_processed = next(json_reader)

#### 데이터셋 확인

In [79]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    10000 non-null  object
 1   user_id      10000 non-null  object
 2   business_id  10000 non-null  object
 3   stars        10000 non-null  int64 
 4   embedding    10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [80]:
df_processed.head(10)

,review_id,user_id,business_id,stars,embedding
0,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4,"[-0.0020696055, -0.0411041267, 0.0311659463, -..."
1,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5,"[-0.0134531558, -0.0046738014, 0.0320035368, -..."
2,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5,"[-0.0221238695, -0.019938929, 0.02893620730000..."
3,cvQXRFLCyr0S7EgFb4lZqw,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5,"[-0.018576180600000002, -0.031470429200000004,..."
4,r2IBPY_E8AE5_GpsqlONyg,IKbjLnfBQtEyVzEu8CuOLg,VJEzpfLs_Jnzgqh5A_FVTg,4,"[-0.008482529800000001, -0.043781627000000004,..."
5,dzNxNW9XpJiECE-bKATezw,NUtIAX-ygn474tDg5nmesg,6LCZLGa09Qifn6rG7-DNrg,4,"[-0.0018611982000000002, -0.0173833352, 0.0224..."
6,DWbmJF84jRrGaJRmlSSnYQ,aWlojpSpzEICTza3RgGJgg,SIoCIxjn4jLt2O-4DajWJw,4,"[-0.0149934087, -0.018939849, 0.03359503670000..."
7,-7LkjSPzfVgnVpuVuRuOow,uAu772KpSkb-tPFgZmU-lA,2GYg3liJ9-m6Z67L_4_BRQ,5,"[-0.0161080286, -0.0323254205, 0.0515194237000..."
8,4KpIldEM-tdnrJLqYzRfZQ,Z5j9Xw_G0c7M2b1-iS67wg,HTqXI5S2XcSlh_ylx9sE6g,5,"[-0.0404551327, -0.0210703239, 0.0208433401, -..."
9,RGV9GWhAAfAAlYyd4vho7g,Zs8Zk3sgh5JxRmoZW4PJcg,3ZynJ94VpIdDlaArmEp2Rg,3,"[-0.0032714282, -0.0399787761, 0.0135638881000..."


In [81]:
print(f"전체 데이터셋 크기: {len(df_processed)}")

전체 데이터셋 크기: 10000


#### user_id, business_id 인코딩

In [82]:
# 각 인코더 객체 생성
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

# 인코딩 수행
encoded_user_ids = user_encoder.fit_transform(df_processed["user_id"])
encoded_business_ids = business_encoder.fit_transform(df_processed["business_id"])

# 데이터프레임에 인코딩된 열 추가
df_processed["user_encoded"] = encoded_user_ids
df_processed["business_encoded"] = encoded_business_ids

In [83]:
# 리뷰 데이터에서 고유한 사용자와 비지니스 수 계산(이후 모델 입력에 사용)

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

print(num_users)
print(num_businesses)

6481
542


#### 데이터셋 학습/검증/테스트 스플릿

In [84]:
# 7:1:2 비율로 데이터셋을 학습, 검증, 테스트로 나누기
# 먼저 학습+검증 / 테스트로 나눔
# 그 후 학습 / 검증으로 나눔

# 학습+검증 / 테스트
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)

# 학습 / 검증
val_size_ratio = 1 / 8  # 전체 데이터의 10% = 학습+검증 데이터의 12.5%
train_df, val_df = train_test_split(
    train_val_df, test_size=val_size_ratio, random_state=42
)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

전체 데이터 수: 10000
학습 데이터 수: 7000 (70.00%)
검증 데이터 수: 1000 (10.00%)
테스트 데이터 수: 2000 (20.00%)


#### 학습을 위해 임베딩 데이터를 numpy 배열로 변환

In [85]:
train_embeddings = np.array(train_df["embedding"].tolist())
val_embeddings = np.array(val_df["embedding"].tolist())
test_embeddings = np.array(test_df["embedding"].tolist())

In [86]:
print(f"학습 임베딩 데이터 형태: {train_embeddings.shape}")
print(f"검증 임베딩 데이터 형태: {val_embeddings.shape}")
print(f"테스트 임베딩 데이터 형태: {test_embeddings.shape}")

print(f"데이터 type: {train_embeddings.dtype}")

학습 임베딩 데이터 형태: (7000, 3072)
검증 임베딩 데이터 형태: (1000, 3072)
테스트 임베딩 데이터 형태: (2000, 3072)
데이터 type: float64


#### 모델 학습에 사용할 하이퍼파라미터 정의

In [87]:
# user_id, business_id의 벡터 차원
user_business_embedding_dim = 64

# 유저-비즈니스 상호작용을 처리하는 MLP의 레이어 크기
user_biz_mlp_dims = [128, 64]

# 제미나이 리뷰 텍스트 임베딩 차원
gemini_embedding_dim = 3072

# 최종 예측을 위한 MLP의 각 레이어 크기
final_mlp_dims = [32, 16]

# 학습률
learning_rate = 0.001

# 배치 사이즈
batch_size = 128
# batch_size = 32

## 모델 정의

#### 1. 유저-비지니스 상호작용 특징 추출 모듈

In [88]:
# 입력층 정의
user_input = keras.Input(shape=(1,), name="user_id_input")
business_input = keras.Input(shape=(1,), name="business_id_input")

# 임베딩 레이어: 각 유저/비즈니스 ID를 고유한 벡터로 변환
user_embedding_layer = layers.Embedding(
    num_users, user_business_embedding_dim, name="user_embedding"
)
business_embedding_layer = layers.Embedding(
    num_businesses, user_business_embedding_dim, name="business_embedding"
)

user_vec = layers.Flatten()(user_embedding_layer(user_input))
business_vec = layers.Flatten()(business_embedding_layer(business_input))

# 두 벡터를 하나로 합침
combined_vec = layers.concatenate([user_vec, business_vec])

# 합쳐진 벡터를 MLP에 통과시켜 상호작용 특징을 추출
interaction_features = combined_vec
for dim in user_biz_mlp_dims:
    interaction_features = layers.Dense(dim, activation="relu")(interaction_features)

#### 2. 리뷰 텍스트(제미나이 임베딩) 특징 추출 모듈

**3072 -> 1536 -> 768 -> 512**

In [89]:
# 입력층 정의
gemini_input = keras.Input(shape=(gemini_embedding_dim,), name="gemini_embedding_input")

# 제미나이 임베딩(리뷰 텍스트)을 처리하는 MLP
review_features = layers.Dense(1536, activation="relu")(gemini_input)
review_features = layers.Dense(768, activation="relu")(review_features)
review_features = layers.Dense(512, activation="relu")(review_features)
# review_features = layers.Dense(256, activation="relu")(review_features)

#### 3. 최종 평점 예측 모듈

In [90]:
# 모듈 1과 모듈 2에서 추출된 특징들을 concat
final_combined_features = layers.concatenate([interaction_features, review_features])

# 최종적으로 별점을 예측하는 MLP
predicted_rating = final_combined_features
for dim in final_mlp_dims:
    predicted_rating = layers.Dense(dim, activation="relu")(predicted_rating)

# 출력층 : 1개의 숫자로 된 최종 별점을 예측
output_rating = layers.Dense(1, activation="linear", name="output_rating")(
    predicted_rating
)

# 최종 모델 정의, 어떤 입력들을 받고 어떤 출력을 내보낼지 설정
final_model = models.Model(
    inputs=[user_input, business_input, gemini_input], outputs=output_rating
)

In [91]:
print("\n--- 생성된 최종 모델 구조 ---")
final_model.summary()


--- 생성된 최종 모델 구조 ---


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_id_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ business_id_input   │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 64)     │    414,784 │ user_id_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ business_embedding  │ (None, 1, 64)     │     34,688 │ business_id_inpu… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 64)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 64)        │          0 │ business_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gemini_embedding_i… │ (None, 3072)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 128)       │          0 │ flatten_4[0][0],  │
│ (Concatenate)       │                   │            │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 1536)      │  4,720,128 │ gemini_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128)       │     16,512 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 768)       │  1,180,416 │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 64)        │      8,256 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 512)       │    393,728 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 576)       │          0 │ dense_16[0][0],   │
│ (Concatenate)       │                   │            │ dense_19[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 32)        │     18,464 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 16)        │        528 │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_rating       │ (None, 1)         │         17 │ dense_21[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,787,521 (25.89 MB)

 Trainable params: 6,787,521 (25.89 MB)

 Non-trainable params: 0 (0.00 B)

----

#### 모델 컴파일

In [92]:
final_model.compile(
    # Adam 옵티마이저
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    # loss 함수 = 평균 제곱 오차 (MSE)
    loss="mse",
    # 학습 중 모니터링할 지표 설정(rmse, mae)
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae"],
)

#### 모델 저장 경로

In [93]:
final_model_path = "final_best_gemini_model.keras"

#### 조기종료, 체크포인트 콜백 정의

In [94]:
early_stopping_callback = callbacks.EarlyStopping(
    monitor="val_rmse",
    patience=10,
    min_delta=0.0005,
    mode="min",
    restore_best_weights=True,
)

model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=final_model_path,
    monitor="val_rmse",
    save_best_only=True,
    mode="min",
    verbose=1,
)

#### 모델 학습

In [95]:
epochs = 50

In [96]:
history = final_model.fit(
    # 입력 데이터
    {
        "user_id_input": train_df["user_encoded"],
        "business_id_input": train_df["business_encoded"],
        "gemini_embedding_input": train_embeddings,
    },
    # 정답 데이터
    train_df["stars"],
    batch_size=batch_size,
    epochs=epochs,
    # 검증 시 사용할 데이터
    validation_data=(
        {
            "user_id_input": val_df["user_encoded"],
            "business_id_input": val_df["business_encoded"],
            "gemini_embedding_input": val_embeddings,
        },
        val_df["stars"],
    ),
    # 콜백 설정
    callbacks=[early_stopping_callback, model_checkpoint_callback],
    verbose=1,
)

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 3.6449 - mae: 1.3898 - rmse: 1.7854
Epoch 1: val_rmse improved from None to 0.52391, saving model to final_best_gemini_model.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - loss: 1.3765 - mae: 0.8032 - rmse: 1.1733 - val_loss: 0.2745 - val_mae: 0.4107 - val_rmse: 0.5239
Epoch 2/50
45/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2946 - mae: 0.4280 - rmse: 0.5426
Epoch 2: val_rmse improved from 0.52391 to 0.50894, saving model to final_best_gemini_model.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.2739 - mae: 0.4114 - rmse: 0.5234 - val_loss: 0.2590 - val_mae: 0.4081 - val_rmse: 0.5089
Epoch 3/50
48/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1869 - mae: 0.3313 - rmse: 0.4319
Epoch 3: val_rmse did not improve from 0.50894
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1769 - mae: 0.3212 - rmse: 0.4205 - val_loss: 0.2713 - val_mae: 0.4139 - val_rmse: 0.5209
Epoch 4/50
46/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 

In [97]:
print(history.history)

{'loss': [1.3765166997909546, 0.27394556999206543, 0.17685411870479584, 0.10724525898694992, 0.07430519908666611, 0.0556204617023468, 0.04007760062813759, 0.02718386985361576, 0.020059658214449883, 0.0161680206656456, 0.014909752644598484, 0.012969616800546646], 'mae': [0.8031713366508484, 0.41142454743385315, 0.32124608755111694, 0.2444949597120285, 0.19701522588729858, 0.1691751778125763, 0.14542189240455627, 0.1202399805188179, 0.10667230933904648, 0.09775455296039581, 0.09447631984949112, 0.08935849368572235], 'rmse': [1.173250436782837, 0.5233981013298035, 0.4205402731895447, 0.32748323678970337, 0.2725898027420044, 0.23583990335464478, 0.20019389688968658, 0.1648753136396408, 0.14163212478160858, 0.12715353071689606, 0.12210549414157867, 0.11388422548770905], 'val_loss': [0.2744773328304291, 0.2590157091617584, 0.27133628726005554, 0.2786838710308075, 0.2852694094181061, 0.2880394756793976, 0.28949904441833496, 0.2961951792240143, 0.2927500605583191, 0.29938560724258423, 0.294900

#### 모델 예측 데이터 생성

In [98]:
final_model = keras.models.load_model(final_model_path)

In [99]:
test_predictions = final_model.predict(
    {
        "user_id_input": test_df["user_encoded"],
        "business_id_input": test_df["business_encoded"],
        "gemini_embedding_input": test_embeddings,
    }
).flatten()

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [100]:
print("예측 결과 샘플")

comparison_df = pd.DataFrame(
    {
        "실제 별점": test_df["stars"].values[:10],
        "예측 별점": test_predictions[:10].round(2),
    }
)

display(comparison_df)

예측 결과 샘플


,실제 별점,예측 별점
0,4,4.61
1,3,3.26
2,1,0.78
3,5,4.41
4,5,4.42
5,4,4.47
6,5,4.54
7,5,4.51
8,3,3.47
9,4,3.55


#### 평가지표 계산

In [101]:
# yelp 평점은 0점이 없으므로 해당 함수 사용 안 함

# # MAPE 오류 방지 함수
# def mean_absolute_percentage_error_test(y_true, y_pred):
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     non_zero_true = y_true != 0
#     if np.sum(non_zero_true) == 0:
#         return 0.0  # 모든 y_true가 0인 경우 MAPE는 0으로 처리
#     return (
#         np.mean(
#             np.abs(
#                 (y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true]
#             )
#         )
#         * 100
#     )

In [102]:
# 테스트 데이터 평점 열을 nparray로 가져옴
true_ratings = test_df["stars"].values

# 각종 평가지표 계산
mse = mean_squared_error(true_ratings, test_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_ratings, test_predictions)
mape = mean_absolute_percentage_error(true_ratings, test_predictions) * 100

# 출력
print(f"최종 모델 성능 평가")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.2f}%")

최종 모델 성능 평가
MSE: 0.2632
RMSE: 0.5131
MAE: 0.4171
MAPE: 12.98%
